In [3]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import numpy as np

In [9]:
def get_routes_and_links(driver):
    """Fetch route names and links from the page."""
    wait = WebDriverWait(driver, 30)
    elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='route']")))
    
    routes, links = [], []
    for element in elements:
        name = element.text
        link = element.get_attribute('href')
        routes.append(name)
        links.append(link)
    
    return routes, links

def navigate_to_next_page(driver, page_number):
    """Navigate to the next page if available."""
    wait = WebDriverWait(driver, 30)
    
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )

        # Scroll to the next page button
        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        time.sleep(1)  # Allow time for scroll

        print(f"Clicking on page {page_number + 1}")
        next_page_button.click()

        # Wait for the page number to update
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)
        ))

        print(f"Successfully navigated to page {page_number + 1}")
        time.sleep(3)  # Wait for the new page to load

    except Exception as e:
        print(f"An error occurred while navigating to page {page_number + 1}: {e}")

def main(url, num_pages):
    """Main function to scrape routes and links."""
    driver = webdriver.Chrome()  # Ensure the correct WebDriver is installed
    driver.get(url)

    all_routes, all_links = [], []

    for page_number in range(1, num_pages + 1):
        routes, links = get_routes_and_links(driver)
        all_routes.extend(routes)
        all_links.extend(links)

        if page_number < num_pages:  # Skip navigation on the last page
            navigate_to_next_page(driver, page_number)

    # Save the results to a CSV file
    df = pd.DataFrame({'Route_Name': all_routes, 'Link1': all_links})
    df.to_csv('kr1.csv', index=False)
    driver.quit()  # Close the driver

if __name__ == "__main__":
    url = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"  # Your target URL
    main(url, 2)  # Scrape data from the first 2 pages

Clicking on page 2
Successfully navigated to page 2


In [10]:
#read the csv file
df1_Ke=pd.read_csv("Kr1.csv")
df1_Ke


,Route_Name,Link1
0,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
5,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
6,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Thiruvananthapuram to Kozhikode,https://www.redbus.in/bus-tickets/thiruvananth...
9,Bangalore to Kannur (Kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [11]:
#retrive the bus details
driver = webdriver.Chrome()
Bus_names_k=[]
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df1_Ke.iterrows():
    link=r["Link1"]
    routes=r["Route_Name"]

# Loop through each link
    driver.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
      
          # click elements to views bus
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))).click()
    except:
        print("No View Button")

    time.sleep(2)


    scrolling = True
    while scrolling:
            old_page_source = driver.page_source  #selenium function
            # Use ActionChains to perform a PAGE_DOWN
            action=ActionChains(driver)
            action.send_keys(Keys.PAGE_DOWN).perform()  
            time.sleep(2)
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
    price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        if ratings.text:
            Ratings_k.append(ratings.text)
        else:
            Ratings_k.append('nan')
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")




Successfully Completed


In [12]:

#  Proceed with creating the DataFrame
data_ke= {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available": Seats_Available_k,
    "Rating":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

# Determine the maximum length
max_len = max(len(values) for values in data_ke.values())

# Normalize the length of each column by padding with NaN
for key in data_ke:
    while len(data_ke[key]) < max_len:
        data_ke[key].append(np.nan)
        
# Create DataFrame
df_buses_1 = pd.DataFrame(data_ke)

# Save to CSV
df_buses_1.to_csv('kr2.csv', index=False)
print("Data saved to kr2.csv successfully!")




Data saved to kr2.csv successfully!


In [13]:
df2_Ke=pd.read_csv("kr2.csv")
df2_Ke

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Rating,Route_link,Route_name
0,KSRTC (Kerala) - 2818,Super Fast Non AC Seater (2+3),15:00,20:30,05h 30m,INR 269,50 Seats available,3.7,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
1,KSRTC (Kerala) - 2810,Super Fast Non AC Seater (2+3),15:25,21:05,05h 40m,INR 269,49 Seats available,3.1,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
2,KSRTC (Kerala) - 720,Super Fast Non AC Seater (2+3),15:40,21:15,05h 35m,INR 269,48 Seats available,2.5,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
3,KSRTC (Kerala) - 2898,Low Floor AC Seater 2+2,16:00,22:15,06h 15m,INR 383,12 Seats available,4.3,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
4,KSRTC (Kerala) - 2874,Super Fast Non AC Seater (2+3),16:20,22:30,06h 10m,INR 230,44 Seats available,2.6,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
...,...,...,...,...,...,...,...,...,...,...
369,Cochin Bus,Bharat Benz A/C Sleeper (2+1),22:15,23:59,01h 44m,INR 700,17 Seats available,4.2,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
370,Cochin Bus,Bharat Benz A/C Semi Sleeper (2+2),22:30,23:59,01h 29m,INR 650,8 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
371,Kottamala Tour And Travels,A/C Semi Sleeper (2+2),18:45,20:45,02h 00m,INR 200,40 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
372,Cochin Bus,A/C Sleeper (2+1),22:40,23:58,01h 18m,INR 800,15 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
